In [1]:
import pandas as pd
import numpy as np

In [2]:
# Main dataset
data_df = pd.read_csv("./Data/dataset.csv")
data_df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Number of target variables
data_df['Disease'].nunique()

41

In [4]:
data_df['Disease'].value_counts()[:10]

Fungal infection                120
Hepatitis C                     120
Hepatitis E                     120
Alcoholic hepatitis             120
Tuberculosis                    120
Common Cold                     120
Pneumonia                       120
Dimorphic hemmorhoids(piles)    120
Heart attack                    120
Varicose veins                  120
Name: Disease, dtype: int64

In [5]:
# All diseases have a sample size of n=120
data_df['Disease'].value_counts().nunique()

1

In [6]:
# Last column contains values
data_df['Symptom_17'].value_counts()

 muscle_pain    72
Name: Symptom_17, dtype: int64

In [78]:
# A more rigorous confirmation that no columns contain only "NaN"
for col in data_df.columns[1:]:
    if data_df[col].value_counts().empty == True:
        print(col)

In [8]:
# Confirm that the .value_counts() of a column with only NaN is empty
test_df = pd.DataFrame({'column': [np.nan] * 3})
print(test_df)

test_df['column'].value_counts().empty

   column
0     NaN
1     NaN
2     NaN


True

In [71]:
# List of symptom column names
symptom_cols = data_df.columns.to_list()[1:]

# Get values from precaution columns as an array
symptom_values = data_df[symptom_cols].values.astype(str)

# Get list of unique values from array
symptom_list = list(np.unique(symptom_values))
symptom_list.remove('nan')
symptom_list[:5]

[' abdominal_pain',
 ' abnormal_menstruation',
 ' acidity',
 ' acute_liver_failure',
 ' altered_sensorium']

In [25]:
# Symptom severity
severity_df = pd.read_csv('./Data/symptom_severity.csv')
severity_df.tail()

,Symptom,weight
127,small_dents_in_nails,2
128,inflammatory_nails,2
129,blister,4
130,red_sore_around_nose,2
131,yellow_crust_ooze,3


In [52]:
# Check number of symptoms in dataset
len(symptom_list)

131

In [70]:
# Check symptoms listed in severity_df
len(severity_df['Symptom'])

132

In [77]:
# Find duplicated symptoms in severity_df
severity_df[severity_df['Symptom'].duplicated()]

,Symptom,weight
117,fluid_overload,4


In [74]:
# Look at both duplicate records
severity_df.loc[severity_df['Symptom'] == 'fluid_overload']

,Symptom,weight
45,fluid_overload,6
117,fluid_overload,4


In [ ]:
# Drop duplicate and replace weight of remaining record with the average, '5'

# severity_df

In [ ]:
# Sort symptoms by weight to view comparatively


In [9]:
# Disease descriptions
desc_df = pd.read_csv('./Data/disease_description.csv')
desc_df.tail()

,Disease,Description
36,Heart attack,The death of heart muscle due to the loss of b...
37,Pneumonia,Pneumonia is an infection in one or both lungs...
38,Arthritis,Arthritis is the swelling and tenderness of on...
39,Gastroenteritis,Gastroenteritis is an inflammation of the dige...
40,Tuberculosis,Tuberculosis (TB) is an infectious disease usu...


In [10]:
# Disease precautions
precaution_df = pd.read_csv('./Data/disease_precaution.csv')
precaution_df.tail()

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
36,Heart attack,call ambulance,chew or swallow asprin,keep calm,NaN
37,Pneumonia,consult doctor,medication,rest,follow up
38,Arthritis,exercise,use hot and cold therapy,try acupuncture,massage
39,Gastroenteritis,stop eating solid food for while,try taking small sips of water,rest,ease back into eating
40,Tuberculosis,cover mouth,consult doctor,medication,rest


In [11]:
# List of precaution column names
precaution_cols = precaution_df.columns.to_list()[1:]

# Get values from precaution columns as an array
precaution_values = precaution_df[precaution_cols].values.astype(str)

# Get list of unique values from array
precaution_list = list(np.unique(precaution_values))
precaution_list

['Consult nearest hospital',
 'acetaminophen',
 'anti itch medicine',
 'antiboitic therapy',
 'apply calamine',
 'avoid abrupt head movment',
 'avoid cold food',
 'avoid fatty spicy food',
 'avoid lying down after eating',
 'avoid non veg food',
 'avoid oily food',
 'avoid open cuts',
 'avoid public places',
 'avoid sudden change in body',
 'avoid too many products',
 'bath twice',
 'call ambulance',
 'check in pulse',
 'chew or swallow asprin',
 'cold baths',
 'consult doctor',
 'consult nearest hospital',
 'consume alovera juice',
 'consume milk thistle',
 'consume neem leaves',
 'consume probiotic food',
 'consume witch hazel',
 'cover area with bandage',
 'cover mouth',
 'dont stand still for long',
 'drink cranberry juice',
 'drink papaya leaf juice',
 'drink plenty of water',
 'drink sugary drinks',
 'drink vitamin c rich drinks',
 'ease back into eating',
 'eat fruits and high fiberous food',
 'eat healthy',
 'eat high calorie vegitables',
 'eliminate milk',
 'exercise',
 'follo

In [13]:
# Explore precaution values - replacement can happen later


In [ ]:
# After cleaning each table, replace Diseases and Symptoms in all tables
# See data_changes.md
